# Water Treatment with SCADA-Driven Performance

Conventional WTP flowsheet + synthetic SCADA (1 year daily) for turbidity, TOC, crypto, DBPs. Runs the model per day, reports removal surrogates, chemical and energy consumption, solids balance, and unit flows.

## Imports and Setup

Requires `watertap`, `pyomo`, `ipopt`, `matplotlib`, `pandas`, `numpy`.

In [3]:
from pyomo.environ import ConcreteModel, TransformationFactory, units as pyunits, value, SolverFactory
from pyomo.network import Arc
from idaes.core import FlowsheetBlock
from idaes.core.util.initialization import propagate_state
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.models.unit_models import Mixer, Separator, Feed, Product
from idaes.models.unit_models.mixer import MomentumMixingType
from watertap.unit_models.clarifier import Clarifier
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock

#!pip show watertap >/dev/null 2>&1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

## Flowsheet Builders

In [6]:
def create_water_treatment_plant():
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    m.fs.properties = NaClParameterBlock()

    m.fs.feed = Feed(property_package=m.fs.properties)
    m.fs.coagulation = Mixer(property_package=m.fs.properties, inlet_list=["water", "coagulant"], momentum_mixing_type=MomentumMixingType.equality)
    m.fs.flocculation_tank = Mixer(property_package=m.fs.properties, inlet_list=["inlet"], momentum_mixing_type=MomentumMixingType.equality)
    m.fs.clarifier = Separator(property_package=m.fs.properties, outlet_list=["effluent", "underflow"])
    m.fs.filtration = Separator(property_package=m.fs.properties, outlet_list=["treated_water", "backwash"])
    m.fs.disinfection = Mixer(property_package=m.fs.properties, inlet_list=["inlet"], momentum_mixing_type=MomentumMixingType.equality)
    m.fs.product = Product(property_package=m.fs.properties)
    m.fs.sludge = Product(property_package=m.fs.properties)

    m.fs.feed_to_coag = Arc(source=m.fs.feed.outlet, destination=m.fs.coagulation.water)
    m.fs.coag_to_floc = Arc(source=m.fs.coagulation.outlet, destination=m.fs.flocculation_tank.inlet)
    m.fs.floc_to_clarifier = Arc(source=m.fs.flocculation_tank.outlet, destination=m.fs.clarifier.inlet)
    m.fs.clarifier_to_filter = Arc(source=m.fs.clarifier.effluent, destination=m.fs.filtration.inlet)
    m.fs.filter_to_disinfect = Arc(source=m.fs.filtration.treated_water, destination=m.fs.disinfection.inlet)
    m.fs.disinfect_to_product = Arc(source=m.fs.disinfection.outlet, destination=m.fs.product.inlet)
    m.fs.clarifier_to_sludge = Arc(source=m.fs.clarifier.underflow, destination=m.fs.sludge.inlet)

    TransformationFactory("network.expand_arcs").apply_to(m)
    return m


def set_operating_conditions(m):
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(100)  # kg/s
    m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(3.5)  # kg/s
    m.fs.feed.properties[0].temperature.fix(298.15)
    m.fs.feed.properties[0].pressure.fix(101325)

    m.fs.coagulation.coagulant.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.5)
    m.fs.coagulation.coagulant.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(0.05)
    m.fs.coagulation.coagulant.temperature[0].fix(298.15)
    m.fs.coagulation.coagulant.pressure[0].fix(101325)

    m.fs.clarifier.split_fraction[0, "effluent"].fix(0.90)
    m.fs.filtration.split_fraction[0, "treated_water"].fix(0.99)


def initialize_model(m):
    m.fs.feed.initialize()
    propagate_state(m.fs.feed_to_coag)
    m.fs.coagulation.initialize(solver='ipopt')
    propagate_state(m.fs.coag_to_floc)
    m.fs.flocculation_tank.initialize(solver='ipopt')
    propagate_state(m.fs.floc_to_clarifier)
    m.fs.clarifier.initialize(solver='ipopt')
    propagate_state(m.fs.clarifier_to_filter)
    propagate_state(m.fs.clarifier_to_sludge)
    m.fs.filtration.initialize(solver='ipopt')
    propagate_state(m.fs.filter_to_disinfect)
    m.fs.disinfection.initialize(solver='ipopt')
    propagate_state(m.fs.disinfect_to_product)


def solve_model(m):
    assert degrees_of_freedom(m) == 0, f"Non-zero DOF: {degrees_of_freedom(m)}"
    solver = SolverFactory('ipopt')
    res = solver.solve(m, tee=False)
    return res


## SCADA Generation and Surrogate Calculations

In [9]:
def generate_scada(seed=42, days=365):
    np.random.seed(seed)
    idx = pd.date_range("2025-01-01", periods=days, freq="D")
    df = pd.DataFrame({
        "Date": idx,
        "Turbidity": np.random.normal(2, 0.5, days).clip(min=0),
        "TOC": np.random.normal(4, 1, days).clip(min=0),
        "Cryptosporidium": np.random.poisson(1, days),
        "DBPs": np.random.normal(60, 10, days).clip(min=0),
        "TDS_mgL": np.random.normal(3500, 300, days).clip(min=500),
    })
    return df


def coagulant_dose_mgL(turbidity, toc):
    return 30 + 5*turbidity + 2*toc


def chlorine_dose_mgL(toc):
    return 2.0 + 0.3*toc


def dose_to_mass_flow_kg_s(dose_mgL, flow_m3_s):
    return dose_mgL * flow_m3_s * 1000 / 1e6


def predict_removals(row):
    turb_rem = np.clip(0.60 + 0.05*row["Turbidity"], 0, 0.98)
    toc_rem = np.clip(0.35 + 0.05*row["TOC"], 0, 0.80)
    crypto_rem = 0.99
    dbp_rem = 0.30
    return {
        "removal_turbidity": turb_rem,
        "removal_TOC": toc_rem,
        "removal_crypto": crypto_rem,
        "removal_dbp": dbp_rem,
    }


def solids_balance(turbidity_ntu, flows, clarifier_split):
    feed_m3_day = flows["flow_feed_m3_s"] * 86400
    tss_mgL = max(turbidity_ntu, 0.1)
    load_kg_day = tss_mgL * feed_m3_day * 1000 / 1e6
    under_frac = value(clarifier_split[0, "underflow"])
    eff_frac = 1 - under_frac
    return {
        "tss_load_kg_day": load_kg_day,
        "tss_effluent_kg_day": load_kg_day * eff_frac,
        "tss_underflow_kg_day": load_kg_day * under_frac,
    }


def estimate_energy_kwh_day(flows):
    m3_day = flows["flow_feed_m3_s"] * 86400
    e_mix = 0.005
    e_pump = 0.01
    e_backwash = 0.002
    total_kwh_day = m3_day * (e_mix + e_pump + e_backwash)
    return {
        "energy_kwh_day": total_kwh_day,
        "energy_kwh_per_m3": (total_kwh_day / m3_day) if m3_day > 0 else float('nan'),
    }


## SCADA Batch Runner

In [12]:
def run_scada_batch(scada_df):
    m = create_water_treatment_plant()
    set_operating_conditions(m)
    initialize_model(m)

    rows = []
    for _, r in scada_df.iterrows():
        # Update feed salt based on TDS (NaCl surrogate)
        na_cl_mass_flow = (r["TDS_mgL"] / 1e6) * value(m.fs.feed.properties[0].flow_vol) * 1000
        m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(na_cl_mass_flow)

        # Chemical dosing
        co_dose = coagulant_dose_mgL(r["Turbidity"], r["TOC"])
        ch_dose = chlorine_dose_mgL(r["TOC"])
        m.fs.coagulation.coagulant.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
            dose_to_mass_flow_kg_s(co_dose, value(m.fs.feed.properties[0].flow_vol))
        )

        solve_model(m)

        flows = {
            "flow_feed_m3_s": value(m.fs.feed.properties[0].flow_vol),
            "flow_coag_m3_s": value(m.fs.coagulation.mixed_state[0].flow_vol),
            "flow_clarifier_eff_m3_s": value(m.fs.clarifier.effluent_state[0].flow_vol),
            "flow_clarifier_und_m3_s": value(m.fs.clarifier.underflow_state[0].flow_vol),
            "flow_filter_eff_m3_s": value(m.fs.filtration.treated_water_state[0].flow_vol),
            "flow_filter_bw_m3_s": value(m.fs.filtration.backwash_state[0].flow_vol),
            "flow_product_m3_s": value(m.fs.product.properties[0].flow_vol),
        }

        removals = predict_removals(r)
        solids = solids_balance(r["Turbidity"], flows, m.fs.clarifier.split_fraction)
        energy = estimate_energy_kwh_day(flows)
        chem = {
            "coagulant_dose_mgL": co_dose,
            "coagulant_kg_day": co_dose * flows["flow_feed_m3_s"] * 86.4 / 1e3,
            "chlorine_dose_mgL": ch_dose,
            "chlorine_kg_day": ch_dose * flows["flow_feed_m3_s"] * 86.4 / 1e3,
        }

        rows.append({**r.to_dict(), **flows, **removals, **solids, **energy, **chem})

    return pd.DataFrame(rows)


## Run SCADA Simulation and Summaries

In [15]:
scada = generate_scada()
results = run_scada_batch(scada)

summary = {
    "avg_product_flow_ML_day": results["flow_product_m3_s"].mean() * 86.4,
    "avg_coagulant_kg_day": results["coagulant_kg_day"].mean(),
    "avg_chlorine_kg_day": results["chlorine_kg_day"].mean(),
    "avg_energy_kwh_per_m3": results["energy_kwh_per_m3"].mean(),
    "avg_tss_load_kg_day": results["tss_load_kg_day"].mean(),
}
summary

2025-11-24 12:30:00 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-11-24 12:30:02 [INFO] idaes.init.fs.coagulation: Initialization Complete: optimal - Optimal Solution Found
2025-11-24 12:30:02 [INFO] idaes.init.fs.flocculation_tank: Initialization Complete: optimal - Optimal Solution Found
2025-11-24 12:30:02 [INFO] idaes.init.fs.clarifier: Initialization Step 2 Complete: optimal - Optimal Solution Found
2025-11-24 12:30:02 [INFO] idaes.init.fs.filtration: Initialization Step 2 Complete: optimal - Optimal Solution Found
2025-11-24 12:30:02 [INFO] idaes.init.fs.disinfection: Initialization Complete: optimal - Optimal Solution Found


AssertionError: Non-zero DOF: -1

## Plots

In [ ]:
results[["flow_feed_m3_s", "flow_product_m3_s"]].mul(86.4).plot(figsize=(9,4), title="Flow (ML/d)")
plt.ylabel("ML/d")
plt.show()

results[["coagulant_kg_day", "chlorine_kg_day"]].plot(figsize=(9,4), title="Chemical Use (kg/d)")
plt.ylabel("kg/d")
plt.show()

results[["energy_kwh_per_m3"]].plot(figsize=(9,3), title="Energy (kWh/m3)")
plt.ylabel("kWh/m3")
plt.show()

results[["tss_effluent_kg_day", "tss_underflow_kg_day"]].plot(figsize=(9,4), title="Solids Split (kg/d)")
plt.ylabel("kg/d")
plt.show()
